In [1]:
import pandas as pd
df = pd.read_csv('cohen_data_annotate.csv')

In [2]:
import json
text = json.load(open('rrcohen.json'))

In [3]:
df['rr'] = df['rr'].apply(json.loads)

In [4]:
ann = df['rr'].apply(lambda s : " ".join([" ".join(x['text']) for x in s['pico_text']]))

In [5]:
asp_rr = {'Population':[], 'Intervention' : [], 'Outcomes' : []}
for r in df['rr'] :
    pt = r['pico_text']
    for a in pt :
        asp_rr[a['domain']].append(" ".join(a['text']))

In [6]:
import spacy
nlp = spacy.load('en')

In [7]:
def tokenizer(sent) :
    text = [token.text for token in nlp(sent)]
    text = ['qqq' if any(char.isdigit() for char in word) else word for word in text]
    return text

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf = TfidfVectorizer(tokenizer=tokenizer, min_df=5)

In [11]:
X_tf = tfidf.fit_transform(ann)

In [18]:
asp_vec = {}
for k in asp_rr :
    tfidf_a = TfidfVectorizer(tokenizer=tokenizer, min_df=2)
    asp_tf = tfidf_a.fit_transform(asp_rr[k])
    asp_vec[k] = asp_tf

In [13]:
from scipy.sparse import hstack
from sklearn.preprocessing import normalize
for k in asp_vec :
    asp_vec[k] = normalize(asp_vec[k], 'l2')
X_tf = hstack(asp_vec.values())

In [14]:
from sklearn.metrics.pairwise import linear_kernel as cosine_similarity
scores = cosine_similarity(X=X_tf)

In [15]:
import numpy as np
nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1

H[np.arange(nb_studies), np.arange(nb_studies)] = 0
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [36]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print(np.mean(aucs))
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.686700594731
{'NSAIDS_processed': 0.74337138475455888, 'ACEInhibitors_processed': 0.67083486778894386, 'SkeletalMuscleRelaxants_processed': 0.56547558067649917, 'Triptans_processed': 0.80534423525497489, 'OralHypoglycemics_processed': 0.716773891692284, 'CalciumChannelBlockers_processed': 0.57539308236954123, 'BetaBlockers_processed': 0.56452966361604173, 'Estrogens_processed': 0.82437828443745609, 'ADHD_processed': 0.84716904503960033, 'Statins_processed': 0.67557677417417661, 'Antihistamines_processed': 0.71702929443082908, 'ProtonPumpInhibitors_processed': 0.67769628791826686, 'Opiods_processed': 0.75935536034572848, 'AtypicalAntipsychotics_processed': 0.62892217075446422}


In [1]:
dic = eval("{'NSAIDS_processed': 0.74337138475455888, 'ACEInhibitors_processed': 0.67083486778894386, 'SkeletalMuscleRelaxants_processed': 0.56547558067649917, 'Triptans_processed': 0.80534423525497489, 'OralHypoglycemics_processed': 0.716773891692284, 'CalciumChannelBlockers_processed': 0.57539308236954123, 'BetaBlockers_processed': 0.56452966361604173, 'Estrogens_processed': 0.82437828443745609, 'ADHD_processed': 0.84716904503960033, 'Statins_processed': 0.67557677417417661, 'Antihistamines_processed': 0.71702929443082908, 'ProtonPumpInhibitors_processed': 0.67769628791826686, 'Opiods_processed': 0.75935536034572848, 'AtypicalAntipsychotics_processed': 0.62892217075446422}")
import pandas as pd
pd.Series(dic)

ACEInhibitors_processed              0.670835
ADHD_processed                       0.847169
Antihistamines_processed             0.717029
AtypicalAntipsychotics_processed     0.628922
BetaBlockers_processed               0.564530
CalciumChannelBlockers_processed     0.575393
Estrogens_processed                  0.824378
NSAIDS_processed                     0.743371
Opiods_processed                     0.759355
OralHypoglycemics_processed          0.716774
ProtonPumpInhibitors_processed       0.677696
SkeletalMuscleRelaxants_processed    0.565476
Statins_processed                    0.675577
Triptans_processed                   0.805344
dtype: float64

1919